# NYC311 Infra Hotspots: End-to-End Vector Search + Geo Clustering in BigQuery AI
----
From millions of 311 tickets to ranked repair polygons with briefs - pure SQL: `ML.GENERATE_EMBEDDING` → `VECTOR_SEARCH` → `ST_CLUSTERDBSCAN` → `ML.GENERATE_TEXT`.
-----
**Author:** Dimitris Lianos  
**Project:** `bq-ai-hackathon-469420`  
**Region:** US multi‑region only  
**Dataset:** `infra_prioritizer_us`

### 3-step Live demo micro-checklist:
Run 1-22 → open 28 in Geo Viz (color by priority_score) → open ops_worklist_with_briefs and read top 5 briefs.

## 0. Executive Summary & Reviewer Guide
**Objective.** Convert raw NYC 311 complaints into ranked, map‑ready infrastructure hotspots using **BigQuery** end‑to‑end: semantic retrieval (embeddings + vector search), per‑category geospatial clustering (DBSCAN), snapshot‑consistent time‑decay scoring, and concise repair briefs (Gemini) - all in the **US** region with cost/QA guards.

**Why this matters.** Field crews need **ranked work areas**, not single tickets. We:
1) retrieve semantically similar complaints per category;  
2) cluster spatially per category;  
3) rank by recency + volume + severity;  
4) output polygons + 4‑bullet repair briefs.

**Auditability & governance.**
- **Regional pinning:** every SQL cell starts with `SET @@location='US';`.  
- **Privacy:** only `ticket_text` is sent to the model; lat/lon stay in BigQuery (used only for clustering/geometry)  
- **Cost control:** 180‑day slice (configurable); `top_k` bounds; optional ANN only when index coverage is healthy. Window is anchored to the source table’s MAX(created_date) to remain snapshot-consistent even if the public dataset isn’t current.
- **QA gates:** weak‑label precision checks per seed; allow‑list for production categories.  
- **Repro:** explicit fully‑qualified identifiers; deterministic temperatures.

**Reviewer quick start.**
- Run **Sections 1 → 22** (core pipeline).  
- Open **Section 27/28** outputs in BigQuery [Geo Viz](https://bigquerygeoviz.appspot.com/) (color by `priority_score`).  
- Read sample **briefs** in **Section 22/22a**.  
- See **Evaluation (Sections 23–25)** for precision/cohesion and QC gating.


Source: [BigQuery Public Datasets — new_york_311.311_service_requests.](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-311)

## 1. One‑time GCP bootstrap (documentation only)
**What/Why.** Verify the BigQuery remote connection and IAM so BigQuery can call Vertex AI. *Reviewers with pre‑wired projects can skip.*

In [100]:
# Documentation commands — intentionally commented
# bq show --location=US --connection bq-ai-hackathon-469420.us.bq_llm_connection
# gcloud services enable aiplatform.googleapis.com --project=bq-ai-hackathon-469420
# gcloud projects add-iam-policy-binding bq-ai-hackathon-469420 \
#   --member="serviceAccount:bqcx-604823017889-tgpd@gcp-sa-bigquery-condel.iam.gserviceaccount.com" \
#   --role="roles/aiplatform.user"

## 2. Environment parameters & dataset creation (Python)
**What.** Centralize project/location/dataset and ensure dataset exists (idempotent).  
**Why.** Single source of truth, US‑only.

In [101]:
# Section 2 — Python env & dataset (US)
PROJECT_ID    = "bq-ai-hackathon-469420"        # if you fork, change these two lines only
LOCATION      = "US"                            # BigQuery multi‑region
DATASET_ID    = "infra_prioritizer_us"
BQ_CONNECTION = "projects/bq-ai-hackathon-469420/locations/us/connections/bq_llm_connection"

from google.cloud import bigquery
print("Using:", PROJECT_ID, LOCATION, DATASET_ID)

client = bigquery.Client(project=PROJECT_ID, location=LOCATION)
ds = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
ds.location = LOCATION
client.create_dataset(ds, exists_ok=True)
print("OK: dataset ready in US")

Using: bq-ai-hackathon-469420 US infra_prioritizer_us
OK: dataset ready in US


### 2a. Global config (parameters in one place)
**What.** Experiment parameters for retrieval, clustering, decay, and QC.  
**Why.** Tuning in one table → consistent downstream behavior.


In [102]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.config` AS
SELECT
  120.0 AS eps_m,           -- DBSCAN epsilon (meters)
  3     AS min_pts,         -- DBSCAN minPts
  30.0  AS half_life_d,     -- time-decay half-life (days)
  14    AS recent14_d,
  30    AS recent30_d,
  200   AS max_clusters,    -- ranked output cap
  400   AS topk_vec,        -- retrieval cap
  0.70  AS p_floor,         -- E1 precision floor for thresholds/QC
  50    AS min_sel,         -- minimum selected items for stable estimates
  50.0  AS ann_cov_min;     -- % coverage required to turn ANN on

""


## 3. Location pin sanity check (must be first per‑SQL cell)
**What/Why.** Guarantees regional compliance.

In [103]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';
SELECT @@location AS job_location;

,job_location
0,US


## 4. Data discovery: NYC 311 schema
**What.** Inspect text and geo columns used by the pipeline.  
**Why.** Avoid brittle assumptions.

In [104]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

SELECT column_name, data_type
FROM `bigquery-public-data.new_york_311`.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '311_service_requests'
ORDER BY ordinal_position;

,column_name,data_type
0,unique_key,INT64
1,created_date,TIMESTAMP
2,closed_date,TIMESTAMP
3,agency,STRING
4,agency_name,STRING
...,...,...
36,road_ramp,STRING
37,bridge_highway_segment,STRING
38,latitude,FLOAT64
39,longitude,FLOAT64


## 5. Sanity skim: recent geo‑tagged tickets
**What.** Quick look at text quality and lat/lon coverage.  
**Why.** Confirm usable inputs before embedding.

In [105]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

SELECT
  created_date, closed_date, complaint_type, descriptor,
  incident_address, city, borough, latitude, longitude, unique_key
FROM `bigquery-public-data.new_york_311.311_service_requests`
WHERE created_date >= TIMESTAMP(DATE_SUB(CURRENT_DATE(), INTERVAL 8 YEAR))
  AND latitude IS NOT NULL AND longitude IS NOT NULL
ORDER BY created_date DESC
LIMIT 10;

,created_date,closed_date,complaint_type,descriptor,incident_address,city,borough,latitude,longitude,unique_key
0,2021-11-08 02:00:12+00:00,NaT,Noise - Residential,Loud Music/Party,50-26 46 STREET,WOODSIDE,QUEENS,40.736746,-73.919990,52448114
1,2021-11-08 01:59:10+00:00,NaT,Noise - Residential,Banging/Pounding,82-01 BRITTON AVENUE,ELMHURST,QUEENS,40.742892,-73.882563,52452478
2,2021-11-08 01:58:02+00:00,NaT,Noise - Commercial,Banging/Pounding,217 NOSTRAND AVENUE,BROOKLYN,BROOKLYN,40.691252,-73.951734,52449210
3,2021-11-08 01:56:14+00:00,NaT,Noise - Residential,Banging/Pounding,1555 EAST 19 STREET,BROOKLYN,BROOKLYN,40.612155,-73.955040,52445913
4,2021-11-08 01:51:10+00:00,NaT,Homeless Person Assistance,Chronic,64-35 59 AVENUE,MASPETH,QUEENS,40.720337,-73.898913,52451792
5,2021-11-08 01:51:07+00:00,NaT,Noise - Residential,Banging/Pounding,4313 FOSTER AVENUE,BROOKLYN,BROOKLYN,40.639431,-73.935150,52449364
6,2021-11-08 01:48:52+00:00,NaT,Noise - Residential,Loud Music/Party,42-30 UNION STREET,FLUSHING,QUEENS,40.757140,-73.824583,52450356
7,2021-11-08 01:48:20+00:00,NaT,Homeless Person Assistance,Chronic,230 PARK AVENUE,NEW YORK,MANHATTAN,40.754617,-73.975864,52453001
8,2021-11-08 01:48:16+00:00,2021-11-08 01:56:55+00:00,Illegal Parking,Posted Parking Sign Violation,306 EAST 49 STREET,NEW YORK,MANHATTAN,40.754050,-73.968526,52454229
9,2021-11-08 01:46:40+00:00,NaT,Noise - Commercial,Loud Music/Party,2241 WASHINGTON AVENUE,BRONX,BRONX,40.854402,-73.893865,52448084


## 6. Build working slice (snapshot‑consistent 180 days)
**What.** Slice using the table’s **own** `MAX(created_date)` to keep decay features consistent on static snapshots.  
**Why.** Reviewers running at different times get the same window relative to the data.

In [106]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_base` AS
WITH bounds AS (
  SELECT DATE(MAX(created_date)) AS max_created_date
  FROM `bigquery-public-data.new_york_311.311_service_requests`
),
base AS (
  SELECT
    unique_key, created_date, closed_date,
    complaint_type, descriptor, incident_address, city, borough,
    latitude, longitude,
    ST_GEOGPOINT(longitude, latitude) AS geom,
    -- PII-safe content passed to embedding model (no street address). reviewer note:
    -- “Only the content column is sent to the model; other columns remain in BigQuery
    -- for joins/geometry.”
    CONCAT(
      COALESCE(complaint_type, ''), ' | ',
      COALESCE(descriptor, ''), ' | ',
      COALESCE(borough, ''), ' | ',
      COALESCE(city, '')
    ) AS ticket_text
  FROM `bigquery-public-data.new_york_311.311_service_requests`
  WHERE latitude IS NOT NULL AND longitude IS NOT NULL
    AND DATE(created_date) >= DATE_SUB((SELECT max_created_date FROM bounds), INTERVAL 180 DAY)
)-- For reviewers: to cut cost/time if regenerating embeddings, change 180→60 days in Sec. 6 and set topk_vec=300 in config
SELECT * FROM base;

""


## 7. Slice QA (row counts & date bounds)
**What.** Guardrails: non‑empty slice; expected time span.  
**Why.** Fail fast if upstream changed.

In [107]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

SELECT
  COUNT(*) AS n_rows,
  MIN(created_date) AS min_created_date,
  MAX(created_date) AS max_created_date
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_base`;

,n_rows,min_created_date,max_created_date
0,1454354,2021-05-12 00:00:00+00:00,2021-11-08 02:00:12+00:00


## 8. Register remote embedding model (US): `text-embedding-005`
**What.** BigQuery remote model via the project connection.  
**Why.** Shared vector space for tickets and seeds.

In [108]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

BEGIN
  CREATE OR REPLACE MODEL `bq-ai-hackathon-469420.infra_prioritizer_us.text_embedding_005`
  REMOTE WITH CONNECTION `projects/bq-ai-hackathon-469420/locations/us/connections/bq_llm_connection`
  OPTIONS (ENDPOINT = 'text-embedding-005');
EXCEPTION WHEN ERROR THEN
  -- If the connection doesn’t exist in the reviewer’s project, continue without failing the notebook.
END;

""


## 9. Active embeddings selection (cached vs fresh)
**What.** Use cached embeddings if present; otherwise generate for current slice.  
**Why.** Keep costs bounded; no manual toggles.

In [109]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

DECLARE has_cached BOOL;

-- Does a cached embedding table exist?
SET has_cached = EXISTS (
  SELECT 1
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us`.INFORMATION_SCHEMA.TABLES
  WHERE table_name = 'tickets_emb'
);

IF has_cached THEN
  -- Use cached embeddings
  EXECUTE IMMEDIATE """
    CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base` AS
    SELECT * FROM `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb`
  """;
ELSE
  -- Build fresh embeddings (uses the remote model from Sec. 8)
  EXECUTE IMMEDIATE """
    CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_fresh` AS
    SELECT
      unique_key, created_date, closed_date,
      complaint_type, descriptor, incident_address, city, borough,
      latitude, longitude, geom,
      ml_generate_embedding_result AS embedding
    FROM ML.GENERATE_EMBEDDING(
      MODEL `bq-ai-hackathon-469420.infra_prioritizer_us.text_embedding_005`,
      (
        SELECT
          unique_key, created_date, closed_date, complaint_type, descriptor,
          incident_address, city, borough, latitude, longitude, geom,
          ticket_text AS content
        FROM `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_base`
      ),
      STRUCT(TRUE AS flatten_json_output)
    )
  """;

  -- Standardize the input name
  EXECUTE IMMEDIATE """
    CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base` AS
    SELECT * FROM `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_fresh`
  """;
END IF;

--Reviewer note: Keep Section 9 commented/unchanged when re‑running to avoid duplicate embedding costs.
--Reviewer note: “Only the content column is sent to the model; other columns
-- remain in BigQuery for joins/geometry.”

""


## 10. Embedding dimensionality sanity check (expect 768)
**What/Why.** Guard against model drift or wrong table.

In [110]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

SELECT ARRAY_LENGTH(embedding) AS dim
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base`
LIMIT 5;

,dim
0,768
1,768
2,768
3,768
4,768


## 11. Compact taxonomy (seed terms)
**What.** Small, interpretable taxonomy → traceable retrieval + per‑category clustering.  
**Why.** Control and audit semantics.

In [111]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_terms` AS
SELECT * FROM UNNEST([
  STRUCT(1 AS seed_id, 'Pothole'            AS seed_label, 'pothole, broken asphalt, road surface hole, roadway cavity, depression in asphalt' AS content),
  STRUCT(2 AS seed_id, 'Cracked road'       AS seed_label, 'cracked roadway, longitudinal cracks, transverse cracks, alligator cracking, asphalt fissures' AS content),
  STRUCT(3 AS seed_id, 'Uneven pavement'    AS seed_label, 'uneven pavement, rutting, heaving, settled area, surface differential, dip, hump, bumpy, sunken' AS content),
  STRUCT(4 AS seed_id, 'Sidewalk damage'    AS seed_label, 'broken sidewalk, lifted slab, trip hazard on sidewalk, cracked sidewalk' AS content),
  STRUCT(5 AS seed_id, 'Sinkhole'           AS seed_label, 'sinkhole, roadway collapse, subsidence, void under pavement, cave-in' AS content),
  STRUCT(6 AS seed_id, 'Manhole cover'      AS seed_label, 'loose manhole cover, missing cover, rattling cover, displaced utility cover' AS content),
  STRUCT(7 AS seed_id, 'Road markings'      AS seed_label, 'faded lane lines, missing crosswalk, worn striping, illegible road markings' AS content),
  STRUCT(8 AS seed_id, 'Street light out'   AS seed_label, 'street light out, lamp failure, outage, dark street lighting' AS content),
  STRUCT(9 AS seed_id, 'Traffic signal'     AS seed_label, 'broken traffic signal, flashing red, signal timing issue, controller fault' AS content),
  STRUCT(10 AS seed_id,'Guardrail damage'   AS seed_label, 'damaged guardrail, bent barrier, missing guardrail section, crash barrier' AS content)
]);

""


## 12. Embed taxonomy seeds (shared vector space)
**What/Why.** Seeds and tickets must share the same model for comparable distances.

In [112]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_terms_emb` AS
SELECT
  seed_id, seed_label,
  ml_generate_embedding_result AS embedding
FROM ML.GENERATE_EMBEDDING(
  MODEL `bq-ai-hackathon-469420.infra_prioritizer_us.text_embedding_005`,
  (SELECT seed_id, seed_label, content AS content
   FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_terms`),
  STRUCT(TRUE AS flatten_json_output)
);

-- Reviewer note: “Only the content column is sent to the model; other columns
-- remain in BigQuery for joins/geometry.”

""


## 13. Semantic retrieval (per‑seed; ANN optional)
**What.** Vector search tickets against seed embeddings with `top_k` cap; if an IVF index exists with sufficient coverage, enable ANN for efficiency.  
**Why.** Precision‑first retrieval before clustering.

In [113]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

DECLARE topk_vec     INT64   DEFAULT (SELECT topk_vec   FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1);
DECLARE ann_fraction FLOAT64 DEFAULT 0.05;
DECLARE ann_cov_min  FLOAT64 DEFAULT (SELECT ann_cov_min FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1);
DECLARE coverage     FLOAT64 DEFAULT 0.0;
DECLARE opts_json    STRING;

SET coverage = COALESCE((
  SELECT coverage_percentage
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us`.INFORMATION_SCHEMA.VECTOR_INDEXES
  WHERE table_name = 'tickets_emb_base' AND index_name LIKE '%tickets_ix%'
  LIMIT 1
), 0.0);

SET opts_json = FORMAT('{"fraction_lists_to_search": %f}', ann_fraction);

IF coverage >= ann_cov_min THEN
  EXECUTE IMMEDIATE """
  CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_all` AS
  SELECT query.seed_label, base.unique_key, base.created_date, base.complaint_type, base.descriptor,
         base.geom, base.latitude, base.longitude, distance, 1.0 - distance AS similarity
  FROM VECTOR_SEARCH(
    TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base`,
    'embedding',
    TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_terms_emb`,
    top_k => @topk_vec,
    distance_type => 'COSINE',
    options => @opts_json
  )""" USING topk_vec AS topk_vec, opts_json AS opts_json;
ELSE
  EXECUTE IMMEDIATE """
  CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_all` AS
  SELECT query.seed_label, base.unique_key, base.created_date, base.complaint_type, base.descriptor,
         base.geom, base.latitude, base.longitude, distance, 1.0 - distance AS similarity
  FROM VECTOR_SEARCH(
    TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base`,
    'embedding',
    TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_terms_emb`,
    top_k => @topk_vec,
    distance_type => 'COSINE'
  )""" USING topk_vec AS topk_vec;
END IF;

""


### 13a. Weak textual hints (expanded coverage)
**What.** RegEx‑based weak labels to estimate **lower‑bound precision** and to guard thresholding.  
**Why.** Avoid inflating precision; support per‑seed tuning.


In [114]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.weak_hints` AS
SELECT
  b.unique_key,
  ANY_VALUE(
    CASE
      WHEN REGEXP_CONTAINS(LOWER(txt), r'\bpotholes?\b|asphalt hole|road (surface )?hole|roadway cavity|asphalt (depression|dip)\b') THEN 'Pothole'

      WHEN REGEXP_CONTAINS(LOWER(txt), r'\brough,\s*pitted\s*or\s*cracked\s*roads?\b')
        OR ( REGEXP_CONTAINS(LOWER(txt), r'\bcrack(ed|s)?\b')
             AND REGEXP_CONTAINS(LOWER(txt), r'\b(road|roadway|asphalt|pavement|street)\b')
             AND NOT REGEXP_CONTAINS(LOWER(txt), r'\bsidewalk\b') )
      THEN 'Cracked road'

      WHEN REGEXP_CONTAINS(LOWER(txt), r'\buneven (road|roadway|pavement|street)\b|\brutt?ing\b|\bheav(?:e|ed|ing)\b|\bsettled\b|\bsunken\b') THEN 'Uneven pavement'

      WHEN REGEXP_CONTAINS(LOWER(txt), r'\bsidewalk\b|trip hazard\b') THEN 'Sidewalk damage'
      WHEN REGEXP_CONTAINS(LOWER(txt), r'\b(sink ?hole|subsidence|collapse|cave[- ]?in)\b') THEN 'Sinkhole'
      WHEN REGEXP_CONTAINS(LOWER(txt), r'\bmanhole\b|\butility cover\b|\b(sewer|utility) lid\b') THEN 'Manhole cover'

      WHEN REGEXP_CONTAINS(LOWER(txt), r'\b(cross ?walk|lane (line|mark)|strip(ing)?|road markings?|stop line|line/marking)\b') THEN 'Road markings'

      WHEN REGEXP_CONTAINS(LOWER(txt), r'\bstreet ?light\b|streetlight\b|\blight out\b|\blighting outage\b') THEN 'Street light out'

      WHEN REGEXP_CONTAINS(LOWER(txt), r'\b(signal|traffic light|controller|timing|flashing|ped(estrian)? signal|walk/do(n\'|)t walk)\b') THEN 'Traffic signal'

      WHEN REGEXP_CONTAINS(LOWER(txt), r'\b(guard[- ]?rail|guide[- ]?rail|guiderail|jersey barrier|crash barrier)\b') THEN 'Guardrail damage'
      ELSE NULL
    END
  ) AS weak_label
FROM (
  SELECT
    unique_key,
    CONCAT(
      COALESCE(complaint_type,''), ' ',
      COALESCE(descriptor,'')
    ) AS txt
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base`
) b
GROUP BY b.unique_key;

""


### 13b. Auto‑tune per‑seed cosine thresholds (optional, default on)
**What.** Pick the **largest** cutoff meeting precision floor & minimum yield; allow manual overrides for problematic seeds.  
**Why.** Precision‑first retrieval prior to clustering.

In [115]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

-- Parameters for tuning
DECLARE p_floor FLOAT64 DEFAULT 0.70;  -- target precision floor
DECLARE min_sel INT64  DEFAULT 50;     -- minimum yield for stability

-- Grid of candidate cosine-distance cutoffs to explore
CREATE TEMP TABLE grid AS
SELECT * FROM UNNEST([0.30,0.33,0.35,0.40,0.45,0.50]) AS max_dist;

-- Sweep precision vs. weak labels
CREATE TEMP TABLE sweep AS
WITH weak AS (SELECT * FROM `bq-ai-hackathon-469420.infra_prioritizer_us.weak_hints`),
scored AS (
  SELECT g.max_dist, m.seed_label, m.unique_key, m.distance, w.weak_label
  FROM grid g
  JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_all` m
    ON m.distance <= g.max_dist
  LEFT JOIN weak w USING (unique_key)
)
SELECT
  seed_label,
  max_dist,
  COUNT(*) AS n_selected,
  SAFE_DIVIDE(COUNTIF(weak_label = seed_label), COUNT(*)) AS precision_est
FROM scored
GROUP BY seed_label, max_dist;

-- Hand overrides for problematic seeds discovered in E1 (stricter)
CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_thresholds_override` AS
SELECT * FROM UNNEST([
  STRUCT('Uneven pavement'  AS seed_label, 0.33 AS max_dist),
  STRUCT('Guardrail damage' AS seed_label, 0.33 AS max_dist)
]);

-- Pick the LARGEST cutoff satisfying both constraints; if none, use smallest grid value.
CREATE TEMP TABLE tuned AS
WITH candidates AS (
  SELECT
    seed_label, max_dist, n_selected, precision_est,
    ROW_NUMBER() OVER (
      PARTITION BY seed_label
      ORDER BY
        CASE WHEN precision_est >= p_floor AND n_selected >= min_sel THEN 1 ELSE 2 END,
        max_dist DESC
    ) AS rn
  FROM sweep
)
SELECT seed_label, max_dist
FROM candidates
WHERE rn = 1;

-- Final thresholds = override if present, else tuned
CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_thresholds` AS
SELECT
  COALESCE(o.seed_label,t.seed_label) AS seed_label,
  COALESCE(o.max_dist, t.max_dist, 0.33) AS max_dist  -- default floor if absent
FROM tuned t
FULL JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.seed_thresholds_override` o
USING (seed_label);

-- Retrieval with thresholds + weak-label guard (keeps precision high)
CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_relaxed` AS
WITH weak AS (SELECT * FROM `bq-ai-hackathon-469420.infra_prioritizer_us.weak_hints`)
SELECT
  st.seed_label, m.unique_key, m.created_date, m.complaint_type, m.descriptor,
  m.geom, m.latitude, m.longitude, m.distance, m.similarity
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_all` m
JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.seed_thresholds` st USING (seed_label)
LEFT JOIN weak w USING (unique_key)
WHERE m.distance <= st.max_dist
  AND (w.weak_label IS NULL OR w.weak_label = st.seed_label);  -- guard against contradictions

""


## 14. Dedupe to best seed per ticket
**What.** When multiple seeds match a ticket, pick the best‑aligned (agree with weak label, then closest distance).  
**Why.** Avoid cross‑category leakage in clustering.

In [116]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_best` AS
WITH scored AS (
  SELECT
    m.seed_label, m.unique_key, m.created_date, m.complaint_type, m.descriptor,
    m.geom, m.latitude, m.longitude, m.distance, m.similarity,
    IF(w.weak_label = m.seed_label, 1, 0) AS agrees_with_weak
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_relaxed` m
  LEFT JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.weak_hints` w USING (unique_key)
)
SELECT * EXCEPT(rn)
FROM (
  SELECT
    scored.*,
    ROW_NUMBER() OVER (
      PARTITION BY unique_key
      ORDER BY agrees_with_weak DESC, distance ASC
    ) AS rn
  FROM scored
)
WHERE rn = 1;

""


## 15. Optional IVF vector index on ticket embeddings
**What.** Create IVF index for ANN; *safe to skip if quota tight*.  
**Why.** Speedup when coverage is healthy.

In [117]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

DECLARE ix_exists BOOL DEFAULT FALSE;

SET ix_exists = EXISTS (
  SELECT 1
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us`.INFORMATION_SCHEMA.VECTOR_INDEXES
  WHERE table_name = 'tickets_emb_base'
);

IF NOT ix_exists THEN
  BEGIN
    EXECUTE IMMEDIATE """
    CREATE VECTOR INDEX `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_ix`
    ON `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base` (embedding)
    OPTIONS(index_type='IVF', distance_type='COSINE', ivf_options='{"num_lists":512}')
    """;
  EXCEPTION WHEN ERROR THEN
    -- Swallow quota errors; ANN is optional for the PoC.
  END;
END IF;

""


### 15a. Index readiness (status only)

ANN auto-enables only if coverage ≥ ann_cov_min (50%).

In [118]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

SELECT
  table_name, index_name, index_status,
  coverage_percentage, last_refresh_time, disable_reason
FROM `bq-ai-hackathon-469420.infra_prioritizer_us`.INFORMATION_SCHEMA.VECTOR_INDEXES
WHERE table_name IN ('tickets_emb_base','tickets_emb')
ORDER BY table_name, index_name;

,table_name,index_name,index_status,coverage_percentage,last_refresh_time,disable_reason
0,tickets_emb,bq-ai-hackathon-469420.infra_prioritizer_us.tickets_ix,ACTIVE,100,2025-09-08 18:09:20.282000+00:00,None
1,tickets_emb_base,bq-ai-hackathon-469420.infra_prioritizer_us.tickets_ix,ACTIVE,0,2025-09-19 11:34:21.803000+00:00,None


### 15b. Optional rebuild/refresh (commented)


In [119]:
# placeholder for refresh script if needed; not required for reviewers

## 16. Cluster hotspots per category with DBSCAN
**What.** Cluster **per seed_label** using `ST_CLUSTERDBSCAN` on geography.  
**Why.** Prevent mixed‑type clusters; meters + minPts tuned via config.

In [120]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

DECLARE eps_m  FLOAT64 DEFAULT 120.0;
DECLARE min_pts INT64  DEFAULT 3;

SET eps_m  = COALESCE((SELECT eps_m  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1), eps_m);
SET min_pts = COALESCE((SELECT min_pts FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1), min_pts);

EXECUTE IMMEDIATE """
CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_clusters_v2` AS
SELECT
  seed_label, unique_key, created_date, complaint_type, descriptor, geom,
  ST_CLUSTERDBSCAN(geom, @eps_m, @min_pts) OVER (PARTITION BY seed_label) AS cluster_id
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_best`
""" USING eps_m AS eps_m, min_pts AS min_pts;

-- ST_CLUSTERDBSCAN uses meters on GEOGRAPHY; eps/minPts tuned in config.


""


## 17. Summarize & rank clusters (snapshot‑consistent decay + weights)
**What.** Score = decayed tickets + recent signals, then multiply by severity weight.  
**Why.** Rank work areas by operational value.

### 17a. Category weights

In [121]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.category_weights` AS
SELECT * FROM UNNEST([
  STRUCT('Sinkhole'         AS seed_label, 2.00 AS w),
  STRUCT('Guardrail damage' AS seed_label, 1.80 AS w),
  STRUCT('Traffic signal'   AS seed_label, 1.50 AS w),
  STRUCT('Manhole cover'    AS seed_label, 1.50 AS w),
  STRUCT('Sidewalk damage'  AS seed_label, 1.20 AS w),
  STRUCT('Pothole'          AS seed_label, 1.20 AS w),
  STRUCT('Uneven pavement'  AS seed_label, 1.10 AS w),
  STRUCT('Road markings'    AS seed_label, 1.00 AS w),
  STRUCT('Cracked road'     AS seed_label, 1.00 AS w),
  STRUCT('Street light out' AS seed_label, 1.00 AS w)
]);

""


### 17b. Decay + ranking

In [122]:
%%bigquery --project bq-ai-hackathon-469420

-- Exact formula:
-- priority_score = (decayed_tickets + 0.5·recent_30 + 1.0·recent_14) × category_weight

SET @@location = 'US';

DECLARE half_life_d FLOAT64 DEFAULT 30.0;
DECLARE recent14_d  INT64   DEFAULT 14;
DECLARE recent30_d  INT64   DEFAULT 30;
DECLARE max_clusters INT64  DEFAULT 200;

SET half_life_d  = COALESCE((SELECT half_life_d  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1), half_life_d);
SET recent14_d   = COALESCE((SELECT recent14_d   FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1), recent14_d);
SET recent30_d   = COALESCE((SELECT recent30_d   FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1), recent30_d);
SET max_clusters = COALESCE((SELECT max_clusters FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1), max_clusters);

EXECUTE IMMEDIATE """
CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.cluster_summary_v2` AS
WITH bounds AS (
  SELECT DATE(MAX(created_date)) AS max_d
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_clusters_v2`
),
decayed AS (
  SELECT
    s.seed_label, s.cluster_id,
    COUNT(*) AS tickets,
    MIN(created_date) AS first_report,
    MAX(created_date) AS last_report,
    COUNTIF(created_date >= TIMESTAMP(DATE_SUB((SELECT max_d FROM bounds), INTERVAL @recent14_d DAY))) AS recent_14,
    COUNTIF(created_date >= TIMESTAMP(DATE_SUB((SELECT max_d FROM bounds), INTERVAL @recent30_d DAY))) AS recent_30,
    ST_CENTROID_AGG(geom) AS center,
    SUM( POW(0.5, GREATEST(0, DATE_DIFF((SELECT max_d FROM bounds), DATE(created_date), DAY)) / @half_life_d) ) AS decayed_tickets,
    ARRAY_AGG(descriptor IGNORE NULLS ORDER BY created_date DESC LIMIT 3) AS sample_descriptors
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_clusters_v2` s
  WHERE cluster_id IS NOT NULL
  GROUP BY seed_label, cluster_id
)
SELECT
  d.seed_label, d.cluster_id, d.tickets, d.recent_14, d.recent_30,
  d.first_report, d.last_report, d.center, d.sample_descriptors,
  ROUND( (d.decayed_tickets + 0.5*d.recent_30 + 1.0*d.recent_14) * w.w, 2) AS priority_score
FROM decayed d
LEFT JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.category_weights` w USING (seed_label)
ORDER BY priority_score DESC
LIMIT @max_clusters
""" USING half_life_d AS half_life_d, recent14_d AS recent14_d, recent30_d AS recent30_d, max_clusters AS max_clusters;

""


## 18. Cluster geometries (convex hulls → polygons) + majority borough
**What.** Build convex hulls; normalize zero‑area hulls with a small buffer; compute majority borough.  
**Why.** Produce stable polygons for Geo Viz.

In [123]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.cluster_geoms_v2` AS
WITH pts AS (
  SELECT sc.seed_label, sc.cluster_id, sc.geom, te.borough
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_clusters_v2` sc
  JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base` te USING (unique_key)
  WHERE sc.cluster_id IS NOT NULL
),
borough_vote AS (
  SELECT seed_label, cluster_id, borough,
         COUNT(*) AS votes,
         ROW_NUMBER() OVER (PARTITION BY seed_label, cluster_id ORDER BY COUNT(*) DESC) AS rn
  FROM pts
  GROUP BY seed_label, cluster_id, borough
),
hulls AS (
  SELECT
    seed_label, cluster_id,
    ST_CONVEXHULL(ST_UNION_AGG(geom)) AS hull_raw,
    ST_CENTROID_AGG(geom)             AS center,
    ARRAY_AGG(geom)                   AS points
  FROM pts
  GROUP BY seed_label, cluster_id
),
normalized AS (
  SELECT
    seed_label,
    cluster_id,
    CASE
      WHEN ST_AREA(hull_raw) = 0 THEN ST_BUFFER(hull_raw, 5)  -- meters → guaranteed polygon
      ELSE hull_raw
    END AS hull_norm,
    center,
    points
  FROM hulls
)
SELECT
  n.seed_label, n.cluster_id,
  n.hull_norm AS hull,
  n.center,
  (SELECT borough FROM borough_vote bv
    WHERE bv.seed_label = n.seed_label AND bv.cluster_id = n.cluster_id AND rn = 1) AS majority_borough,
  ST_AREA(n.hull_norm) AS area_m2,
  ARRAY_LENGTH(n.points) AS tickets_in_cluster
FROM normalized n;

""


## 19. Final prioritized hotspots (ranked, map‑ready)
**What.** Join scores with geometries; produce primary artifact for mapping and briefs.  
**Why.** Single truth for ops and reviewers.

In [124]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2` AS
SELECT
  s.seed_label, s.cluster_id, s.tickets, s.recent_14, s.recent_30,
  s.first_report, s.last_report, s.priority_score,
  g.majority_borough AS borough, s.center, g.hull, g.area_m2, s.sample_descriptors
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.cluster_summary_v2` s
JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.cluster_geoms_v2` g USING (seed_label, cluster_id)
ORDER BY priority_score DESC;

""


## 20. Register remote text model (US): `gemini-2.0-flash-001`
**What/Why.** Deterministic low‑temperature summarization into 4 bullets.

In [125]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

BEGIN
  CREATE OR REPLACE MODEL `bq-ai-hackathon-469420.infra_prioritizer_us.gemini_text`
  REMOTE WITH CONNECTION `projects/bq-ai-hackathon-469420/locations/us/connections/bq_llm_connection`
  OPTIONS (ENDPOINT = 'gemini-2.0-flash-001');
EXCEPTION WHEN ERROR THEN
END;

""


### 21. Text model smoke test

This verifies the remote Gemini model over a BigQuery connection.

In [126]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

SELECT ml_generate_text_llm_result AS out
FROM ML.GENERATE_TEXT(
  MODEL `bq-ai-hackathon-469420.infra_prioritizer_us.gemini_text`,
  (SELECT 'Reply with: Gemini is here' AS prompt),
  STRUCT(TRUE AS flatten_json_output, 64 AS max_output_tokens, 0.2 AS temperature)
);

,out
0,Gemini is here\n


## 22. Generate concise repair briefs (4 bullets, <80 words)
**What.** Create `hotspot_briefs` for top N hotspots with strict prompt.

In [127]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

BEGIN
  CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.hotspot_briefs` AS
  SELECT
    seed_label, cluster_id, borough, priority_score,
    ST_X(center) AS lon, ST_Y(center) AS lat,
    ml_generate_text_llm_result AS brief
  FROM ML.GENERATE_TEXT(
    MODEL `bq-ai-hackathon-469420.infra_prioritizer_us.gemini_text`,
    (
      SELECT
        CONCAT(
          'Return exactly 4 bullets; no intro/closing. Do NOT invent street names. ',
          'Use borough only, Proper Case. ',
          'Borough: ', INITCAP(COALESCE(borough,'(unknown)')), '. Category: ', seed_label, '. ',
          'Tickets: ', CAST(tickets AS STRING), '. Recent14: ', CAST(recent_14 AS STRING),
          ', Recent30: ', CAST(recent_30 AS STRING), '. ',
          CASE
            WHEN seed_label IN ('Sidewalk damage','Pothole','Cracked road','Uneven pavement','Road markings')
            THEN CONCAT('Approx area (m^2): ', CAST(ROUND(area_m2) AS STRING), '. ')
            ELSE ''
          END,
          'Descriptors (sample): ', ARRAY_TO_STRING(sample_descriptors, '; '), '. ',
          'Bullets: ',
          '(1) One-line issue summary with borough and approx. span if obvious; ',
          '(2) Most likely fix and a rough unit (m² or linear m) IF inferable from category only; ',
          '(3) Urgency Low/Medium/High with reason (recency/volume); ',
          '(4) Crew type + 1–2 key equipment. ',
          'Hard cap 80 words total. Output ONLY bullets, each starting with "- ".'
        ) AS prompt,
        seed_label, cluster_id, borough, priority_score, center
      FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2`
      ORDER BY priority_score DESC
      LIMIT 200
    ),
    STRUCT(TRUE AS flatten_json_output, 200 AS max_output_tokens, 0.1 AS temperature)
  );
EXCEPTION WHEN ERROR THEN
  CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.hotspot_briefs` AS
  SELECT
    seed_label, cluster_id, borough, priority_score,
    ST_X(center) AS lon, ST_Y(center) AS lat,
    '- Briefs unavailable (LLM connection missing in this project).' AS brief
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2`
  ORDER BY priority_score DESC
  LIMIT 200;
END;

""


### 22a. Preview top briefs with context (spot check)

In [128]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

SELECT
  h.seed_label, h.cluster_id, h.borough, ROUND(h.priority_score, 1) AS score,
  p.tickets, p.recent_14, p.recent_30,
  ST_Y(p.center) AS lat, ST_X(p.center) AS lon,
  p.sample_descriptors,
  h.brief
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.hotspot_briefs` h
JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2` p
  USING (seed_label, cluster_id, borough, priority_score)
ORDER BY score DESC;

,seed_label,cluster_id,borough,score,tickets,recent_14,recent_30,lat,lon,sample_descriptors,brief
0,Sidewalk damage,1,QUEENS,19.2,7,7,7,40.772839,-73.771485,"[Broken Sidewalk, Broken Sidewalk, Broken Sidewalk]",- Queens: Extensive sidewalk damage spanning approximately 15088 m².\n- Likely fix: Sidewalk replacement; approx. 15...
1,Sinkhole,8,MANHATTAN,11.5,3,2,3,40.811477,-73.950216,"[Cave-in, Cave-in, Cave-in]","- Manhattan sinkhole, possibly small.\n- Fill and compact. ~1 m².\n- Urgency: Medium. Recent volume and recency sugg..."
2,Sidewalk damage,11,MANHATTAN,9.8,4,3,4,40.726654,-73.991713,"[Broken Sidewalk, Broken Sidewalk, Broken Sidewalk]","- Sidewalk damage reported in Manhattan, approx. 220 m².\n- Repair/replace damaged sidewalk slabs; approx. 220 m².\n..."
3,Street light out,7,QUEENS,9.3,4,4,4,40.703010,-73.799798,"[Street Light Out, Street Light Out, Street Light Out]","- Queens: Street light out, affecting visibility on a block.\n- Likely fix: Bulb replacement. Unit: 1 light fixture...."
4,Street light out,6,QUEENS,8.4,4,3,4,40.697482,-73.836212,"[Street Light Out, Street Light Out, Street Light Out]","- Queens: Street light out, affecting immediate area.\n- Likely bulb/fixture replacement; illumination area ~50 m².\..."
...,...,...,...,...,...,...,...,...,...,...,...
175,Road markings,2,STATEN ISLAND,0.1,3,0,0,40.544827,-74.165544,"[Line/Marking - Faded, Line/Marking - Faded, Line/Marking - Faded]","- Staten Island: Faded road markings, approx. 51 m².\n- Repainting of road markings, approx. 51 m².\n- Urgency: Low...."
176,Sinkhole,5,QUEENS,0.1,4,0,0,40.732662,-73.868204,"[Cave-in, Cave-in, Cave-in]",- Sinkhole reported in Queens.\n- Likely requires backfilling and asphalt patching; approx. 5 m².\n- Urgency Low; no...
177,Road markings,19,STATEN ISLAND,0.1,3,0,0,40.630318,-74.086821,"[Line/Marking - Faded, Line/Marking - Faded, Line/Marking - Faded]","- Staten Island: Faded road markings, approx. 78 m².\n- Repaint road markings; approx. 78 m².\n- Urgency: Low. No re..."
178,Road markings,24,STATEN ISLAND,0.1,3,0,0,40.513457,-74.237400,"[Line/Marking - Faded, Line/Marking - Faded, Line/Marking - Faded]","- Staten Island: Faded road markings, approx. 340 m².\n- Repaint road markings; approx. 340 m² of paint.\n- Urgency:..."


## 23. Evaluation & Diagnostics (Reviewer‑Focused)
**Goal.** Provide quantitative signals that (a) semantic retrieval is precise and stable, (b) clusters are cohesive/reasonable, and (c) outputs are map/ops‑ready.

### 23a. E0 — distributional checks

In [129]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

-- Matches by seed label
SELECT seed_label, COUNT(*) AS matches
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_relaxed`
GROUP BY seed_label
ORDER BY matches DESC;

-- Top seed×borough combos
SELECT sm.seed_label, te.borough, COUNT(*) AS matches
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_relaxed` sm
JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base` te USING (unique_key)
GROUP BY seed_label, borough
ORDER BY matches DESC
LIMIT 200;

,seed_label,borough,matches
0,Pothole,MANHATTAN,400
1,Street light out,QUEENS,400
2,Road markings,STATEN ISLAND,268
3,Traffic signal,MANHATTAN,214
4,Sinkhole,QUEENS,167
...,...,...,...
26,Sinkhole,STATEN ISLAND,14
27,Sidewalk damage,BRONX,8
28,Road markings,BRONX,8
29,Sidewalk damage,STATEN ISLAND,5


### 23b. E2 — cluster cohesion & stability sweep

In [130]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

DECLARE eps_cfg  FLOAT64 DEFAULT (SELECT eps_m  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1);
DECLARE mp_cfg   INT64   DEFAULT (SELECT min_pts FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1);

WITH
r0 AS (
  SELECT FORMAT('eps=%.0f,minPts=%d', eps_cfg, mp_cfg) AS setting, seed_label,
         ST_CLUSTERDBSCAN(geom, eps_cfg, mp_cfg) OVER (PARTITION BY seed_label) AS cid, geom
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_best`
),
r1 AS (
  SELECT 'eps=100,minPts=3' AS setting, seed_label,
         ST_CLUSTERDBSCAN(geom, 100.0, 3) OVER (PARTITION BY seed_label) AS cid, geom
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_best`
),
r2 AS (
  SELECT 'eps=150,minPts=3' AS setting, seed_label,
         ST_CLUSTERDBSCAN(geom, 150.0, 3) OVER (PARTITION BY seed_label) AS cid, geom
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_best`
),
r3 AS (
  SELECT 'eps=200,minPts=4' AS setting, seed_label,
         ST_CLUSTERDBSCAN(geom, 200.0, 4) OVER (PARTITION BY seed_label) AS cid, geom
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_best`
),
all_rows AS (
  SELECT * FROM r0 UNION ALL SELECT * FROM r1 UNION ALL SELECT * FROM r2 UNION ALL SELECT * FROM r3
),
centroids AS (
  SELECT setting, seed_label, cid, ST_CENTROID_AGG(geom) AS cgeom
  FROM all_rows
  WHERE cid IS NOT NULL
  GROUP BY setting, seed_label, cid
),
cohesion AS (
  SELECT a.setting, a.seed_label, a.cid,
         AVG(ST_DISTANCE(a.geom, c.cgeom)) AS mean_intra_m
  FROM all_rows a
  JOIN centroids c ON a.setting=c.setting AND a.seed_label=c.seed_label AND a.cid=c.cid
  WHERE a.cid IS NOT NULL
  GROUP BY 1,2,3
)
SELECT
  setting,
  COUNT(DISTINCT CONCAT(seed_label, ':', cid)) AS n_clusters,
  SUM(CASE WHEN cid IS NULL THEN 1 ELSE 0 END) AS noise_pts,
  ROUND(AVG(mean_intra_m),1) AS avg_intra_m
FROM all_rows
LEFT JOIN cohesion USING (setting, seed_label, cid)
GROUP BY setting
ORDER BY setting;

,setting,n_clusters,noise_pts,avg_intra_m
0,"eps=100,minPts=3",178,2391,23.1
1,"eps=120,minPts=3",180,2362,28.0
2,"eps=150,minPts=3",193,2279,38.8
3,"eps=200,minPts=4",117,2389,84.8


### 23c. E1 — retrieval precision @200 (vs. weak labels)

In [131]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

WITH cfg AS (
  SELECT
    COALESCE((SELECT p_floor FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config`), 0.70) AS p_floor,
    COALESCE((SELECT min_sel FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config`), 50)    AS min_sel
),
weak AS (
  SELECT * FROM `bq-ai-hackathon-469420.infra_prioritizer_us.weak_hints`
),
top_vec AS (
  -- IMPORTANT: evaluate on raw retrieval, not thresholded set
  SELECT seed_label, unique_key, distance
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_all`
  QUALIFY ROW_NUMBER() OVER(PARTITION BY seed_label ORDER BY distance ASC) <= 200
),
scored AS (
  SELECT v.seed_label,
         SAFE_DIVIDE(COUNTIF(w.weak_label = v.seed_label), COUNT(*)) AS precision_at_200,
         COUNT(*) AS n_eval
  FROM top_vec v
  LEFT JOIN weak w USING (unique_key)
  GROUP BY v.seed_label
)
SELECT
  s.seed_label, s.precision_at_200, s.n_eval,
  (SELECT p_floor FROM cfg) AS p_floor,
  (SELECT min_sel  FROM cfg) AS min_sel
FROM scored s
ORDER BY s.seed_label;

,seed_label,precision_at_200,n_eval,p_floor,min_sel
0,Cracked road,0.98,200,0.7,50
1,Guardrail damage,0.35,200,0.7,50
2,Manhole cover,1.00,200,0.7,50
3,Pothole,1.00,200,0.7,50
4,Road markings,1.00,200,0.7,50
5,Sidewalk damage,1.00,200,0.7,50
6,Sinkhole,1.00,200,0.7,50
7,Street light out,1.00,200,0.7,50
8,Traffic signal,1.00,200,0.7,50
9,Uneven pavement,0.01,200,0.7,50


### 23d. E1 extended — per‑seed threshold curves

In [132]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

WITH grid AS (
  SELECT * FROM UNNEST([0.30,0.33,0.35,0.40,0.45,0.50,0.55]) AS max_dist
),
weak AS (
  SELECT * FROM `bq-ai-hackathon-469420.infra_prioritizer_us.weak_hints`
),
scored AS (
  SELECT g.max_dist, m.seed_label, m.unique_key, m.distance, w.weak_label
  FROM grid g
  JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_all` m   -- raw retrieval
    ON m.distance <= g.max_dist
  LEFT JOIN weak w USING (unique_key)
)
SELECT
  seed_label,
  max_dist,
  COUNT(*) AS n_selected,
  SAFE_DIVIDE(COUNTIF(weak_label = seed_label), COUNT(*)) AS precision_est
FROM scored
GROUP BY seed_label, max_dist
ORDER BY seed_label, max_dist;

,seed_label,max_dist,n_selected,precision_est
0,Cracked road,0.35,26,0.884615
1,Cracked road,0.40,400,0.940000
2,Cracked road,0.45,400,0.940000
3,Cracked road,0.50,400,0.940000
4,Cracked road,0.55,400,0.940000
...,...,...,...,...
61,Uneven pavement,0.35,124,0.008065
62,Uneven pavement,0.40,400,0.005000
63,Uneven pavement,0.45,400,0.005000
64,Uneven pavement,0.50,400,0.005000


## 24. Quality gate — allow‑list seeds meeting precision bar
**What.** Exclude categories failing configured precision floor or minimum yield.  
**Why.** Demo‑safe artifacts only.

In [133]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

DECLARE p_floor FLOAT64 DEFAULT COALESCE((SELECT p_floor FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config`), 0.70);
DECLARE min_sel INT64   DEFAULT COALESCE((SELECT min_sel  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config`), 50);

-- Compute precision and yield at the chosen thresholds
CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_precision_at_threshold` AS
WITH weak AS (
  SELECT * FROM `bq-ai-hackathon-469420.infra_prioritizer_us.weak_hints`
),
joined AS (
  SELECT r.seed_label, r.unique_key, r.distance, w.weak_label
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_relaxed` r
  LEFT JOIN weak w USING (unique_key)
)
SELECT
  seed_label,
  COUNT(*) AS n_selected,
  SAFE_DIVIDE(COUNTIF(weak_label = seed_label), COUNT(*)) AS precision_est
FROM joined
GROUP BY seed_label;

-- Only keep seeds that meet the floor and have enough support
CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_allowlist` AS
SELECT
  seed_label, precision_est, n_selected,
  TRUE AS allow
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_precision_at_threshold`
WHERE precision_est IS NOT NULL
  AND precision_est >= p_floor
  AND n_selected   >= min_sel;

""


### 24a. Apply gating to prioritized output

In [134]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc` AS
SELECT p.*
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2` p
JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.seed_allowlist` a USING (seed_label)
WHERE a.allow;

""


### 24b. Production allow-list (proof)


In [135]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';
SELECT seed_label, n_selected, ROUND(precision_est,2) AS precision_est
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.seed_allowlist`
ORDER BY precision_est DESC;

,seed_label,n_selected,precision_est
0,Traffic signal,400,1.00
1,Sidewalk damage,400,1.00
2,Sinkhole,400,1.00
3,Manhole cover,400,1.00
4,Street light out,400,1.00
5,Road markings,400,1.00
6,Pothole,400,1.00
7,Cracked road,392,0.96


### 24c. Assert QC table only contains allow-listed seeds (should return 0 rows)

In [136]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';
SELECT DISTINCT p.seed_label
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc` p
LEFT JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.seed_allowlist` a USING (seed_label)
WHERE a.allow IS NULL;

,seed_label


## 25. QC‑filtered briefs (optional — mirrors Section 22 on QC set)

In [137]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

BEGIN
  CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.hotspot_briefs_qc` AS
  SELECT
    seed_label, cluster_id, borough, priority_score,
    ST_X(center) AS lon, ST_Y(center) AS lat,
    ml_generate_text_llm_result AS brief
  FROM ML.GENERATE_TEXT(
    MODEL `bq-ai-hackathon-469420.infra_prioritizer_us.gemini_text`,
    (
      SELECT
        CONCAT(
          'Return exactly 4 bullets; no intro/closing. Do NOT invent street names. ',
          'Use borough only, Proper Case. ',
          'Borough: ', INITCAP(COALESCE(borough,'(unknown)')), '. Category: ', seed_label, '. ',
          'Tickets: ', CAST(tickets AS STRING), '. Recent14: ', CAST(recent_14 AS STRING),
          ', Recent30: ', CAST(recent_30 AS STRING), '. ',
          CASE
            WHEN seed_label IN ('Sidewalk damage','Pothole','Cracked road','Uneven pavement','Road markings')
            THEN CONCAT('Approx area (m^2): ', CAST(ROUND(area_m2) AS STRING), '. ')
            ELSE ''
          END,
          'Descriptors (sample): ', ARRAY_TO_STRING(sample_descriptors, '; '), '. ',
          'Bullets: ',
          '(1) One-line issue summary with borough and approx. span if obvious; ',
          '(2) Most likely fix and a rough unit (m² or linear m) IF inferable from category only; ',
          '(3) Urgency Low/Medium/High with reason (recency/volume); ',
          '(4) Crew type + 1–2 key equipment. ',
          'Hard cap 80 words total. Output ONLY bullets, each starting with "- ".'
        ) AS prompt,
        seed_label, cluster_id, borough, priority_score, center
      FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc`
      ORDER BY priority_score DESC
      LIMIT 200
    ),
    STRUCT(TRUE AS flatten_json_output, 200 AS max_output_tokens, 0.1 AS temperature)
  );
EXCEPTION WHEN ERROR THEN
  CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.hotspot_briefs_qc` AS
  SELECT
    seed_label, cluster_id, borough, priority_score,
    ST_X(center) AS lon, ST_Y(center) AS lat,
    '- Briefs unavailable (LLM connection missing in this project).' AS brief
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc`
  ORDER BY priority_score DESC
  LIMIT 200;
END;

""


## 26. Publish ops view (QC‑filtered ranked worklist + briefs)
**What.** Single view for dispatch with polygons, scores, and briefs.  
**Why.** Ready for export or BI wiring.

In [138]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

DECLARE has_ph BOOL DEFAULT FALSE;
DECLARE has_briefs BOOL DEFAULT FALSE;

SET has_ph = EXISTS (
  SELECT 1 FROM `bq-ai-hackathon-469420.infra_prioritizer_us`.INFORMATION_SCHEMA.TABLES
  WHERE table_name = 'prioritized_hotspots_v2_qc'
);

SET has_briefs = EXISTS (
  SELECT 1 FROM `bq-ai-hackathon-469420.infra_prioritizer_us`.INFORMATION_SCHEMA.TABLES
  WHERE table_name = 'hotspot_briefs_qc'
);

IF has_ph AND has_briefs THEN
  EXECUTE IMMEDIATE """
  CREATE OR REPLACE VIEW `bq-ai-hackathon-469420.infra_prioritizer_us.ops_worklist_with_briefs` AS
  SELECT
    ph.seed_label, ph.cluster_id, ph.borough,
    ph.tickets, ph.recent_14, ph.recent_30, ph.priority_score,
    ST_Y(ph.center) AS lat, ST_X(ph.center) AS lon,
    ph.hull, ph.sample_descriptors, hb.brief
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc` ph
  LEFT JOIN `bq-ai-hackathon-469420.infra_prioritizer_us.hotspot_briefs_qc` hb
    USING (seed_label, cluster_id, borough, priority_score)
  ORDER BY priority_score DESC
  """;
ELSEIF has_ph THEN
  EXECUTE IMMEDIATE """
  CREATE OR REPLACE VIEW `bq-ai-hackathon-469420.infra_prioritizer_us.ops_worklist_with_briefs` AS
  SELECT
    ph.seed_label, ph.cluster_id, ph.borough,
    ph.tickets, ph.recent_14, ph.recent_30, ph.priority_score,
    ST_Y(ph.center) AS lat, ST_X(ph.center) AS lon,
    ph.hull, ph.sample_descriptors,
    CAST(NULL AS STRING) AS brief
  FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc` ph
  ORDER BY priority_score DESC
  """;
END IF;

""


## 27. Map‑friendly table (QC‑filtered, GeoJSON center + hull)
**What.** Exportable table for Geo Viz / BI.  
**Why.** Make mapping trivial.

In [139]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

CREATE OR REPLACE TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.map_hotspots_v2` AS
SELECT
  seed_label, cluster_id, borough,
  tickets, recent_14, recent_30, priority_score,
  ST_Y(center) AS lat, ST_X(center) AS lon,
  ST_ASGEOJSON(hull)   AS hull_geojson,
  ST_ASGEOJSON(center) AS center_geojson
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc`
ORDER BY priority_score DESC;

""


## 28. GeoJSON export for map preview (QC‑filtered)
**What.** Copy/paste this query’s results into **BigQuery Geo Viz** (color by `priority_score`).

In [140]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

DECLARE max_clusters INT64 DEFAULT 200;
SET max_clusters = COALESCE(
  (SELECT max_clusters FROM `bq-ai-hackathon-469420.infra_prioritizer_us.config` LIMIT 1),
  max_clusters
);

EXECUTE IMMEDIATE """
SELECT
  seed_label, cluster_id, borough,
  tickets, recent_14, recent_30, priority_score,
  ST_Y(center) AS lat, ST_X(center) AS lon,
  ST_ASGEOJSON(hull) AS hull_geojson,
  area_m2,
  sample_descriptors
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc`
ORDER BY priority_score DESC
LIMIT @max_clusters
""" USING max_clusters AS max_clusters;

,seed_label,cluster_id,borough,tickets,recent_14,recent_30,priority_score,lat,lon,hull_geojson,area_m2,sample_descriptors
0,Sidewalk damage,1,QUEENS,7,7,7,19.16,40.772839,-73.771485,"{ ""type"": ""Polygon"", ""coordinates"": [ [ [-73.7725856315463, 40.7735563859716], [-73.7718956174221, 40.7726245525999]...",15087.956182,"[Broken Sidewalk, Broken Sidewalk, Broken Sidewalk]"
1,Sinkhole,8,MANHATTAN,3,2,3,11.48,40.811477,-73.950216,"{ ""type"": ""Polygon"", ""coordinates"": [ [ [-73.9502056729149, 40.8115214062634], [-73.9502172817109, 40.8115221158121]...",78.036129,"[Cave-in, Cave-in, Cave-in]"
2,Sidewalk damage,11,MANHATTAN,4,3,4,9.83,40.726654,-73.991713,"{ ""type"": ""Polygon"", ""coordinates"": [ [ [-73.9917808388347, 40.7266345471319], [-73.9917827099019, 40.7266258470453]...",220.140755,"[Broken Sidewalk, Broken Sidewalk, Broken Sidewalk]"
3,Street light out,7,QUEENS,4,4,4,9.35,40.703010,-73.799798,"{ ""type"": ""Polygon"", ""coordinates"": [ [ [-73.800203189788, 40.7033417498033], [-73.7999208696955, 40.7024739113054],...",4830.758495,"[Street Light Out, Street Light Out, Street Light Out]"
4,Street light out,6,QUEENS,4,3,4,8.38,40.697482,-73.836212,"{ ""type"": ""Polygon"", ""coordinates"": [ [ [-73.8371654365109, 40.6968646746722], [-73.8356412062153, 40.6978286678371]...",394.823475,"[Street Light Out, Street Light Out, Street Light Out]"
...,...,...,...,...,...,...,...,...,...,...,...,...
175,Cracked road,0,MANHATTAN,3,0,0,0.09,40.841077,-73.939767,"{ ""type"": ""Polygon"", ""coordinates"": [ [ [-73.9397568236059, 40.8411209240157], [-73.9397684373659, 40.8411216356097]...",78.036129,"[Failed Street Repair, Failed Street Repair, Failed Street Repair]"
176,Pothole,0,MANHATTAN,3,0,0,0.08,40.777647,-73.951908,"{ ""type"": ""Polygon"", ""coordinates"": [ [ [-73.9516913896678, 40.777852380742], [-73.9516867982754, 40.7778604809776],...",941.968582,"[Pothole, Pothole, Pothole]"
177,Sidewalk damage,7,QUEENS,3,0,0,0.07,40.681882,-73.752059,"{ ""type"": ""Polygon"", ""coordinates"": [ [ [-73.7523127527626, 40.6813947201276], [-73.7519605144732, 40.6820691781349]...",1.902199,"[Broken Sidewalk, Broken Sidewalk, Broken Sidewalk]"
178,Road markings,19,STATEN ISLAND,3,0,0,0.05,40.630318,-74.086821,"{ ""type"": ""Polygon"", ""coordinates"": [ [ [-74.0868107527632, 40.6303618568737], [-74.0868223321129, 40.6303625463502]...",78.036129,"[Line/Marking - Faded, Line/Marking - Faded, Line/Marking - Faded]"


### 28a. Sanity check (no zero‑area hulls after normalization)

In [141]:
%%bigquery --project bq-ai-hackathon-469420
SET @@location = 'US';

SELECT
  COUNTIF(ST_AREA(hull) = 0) AS zero_area_hulls,
  COUNT(*) AS total_clusters
FROM `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc`;

,zero_area_hulls,total_clusters
0,0,180


## 29. Evaluation summary & reviewer pointers
- **E1 (Precision@200):** production categories meet ≥ `p_floor`; failing seeds are excluded by the **allow‑list** (Sec. 24).
- Uneven pavement and Guardrail damage are excluded unless they meet the precision floor at tuned thresholds.  
- **E1 curves:** Sec. 23d justifies conservative `max_dist` (e.g., stricter for **Guardrail damage** and **Uneven pavement** at `0.33`).  
- **E2 (Cohesion/Noise):** Sec. 23b shows stability across nearby DBSCAN settings.  
- **Ops readiness:** `prioritized_hotspots_v2_qc` + `hotspot_briefs_qc` are demo‑safe; `map_hotspots_v2` supports Geo Viz.

If any seed underperforms: (a) tighten its threshold, (b) expand seed phrasings, (c) enrich weak‑label synonyms.


## 30. Submission artifacts & reproducibility notes
**Tables**  
- `infra_prioritizer_us.prioritized_hotspots_v2`  
- `infra_prioritizer_us.prioritized_hotspots_v2_qc`  
- `infra_prioritizer_us.map_hotspots_v2`

**View**  
- `infra_prioritizer_us.ops_worklist_with_briefs`

**Briefs**  
- `infra_prioritizer_us.hotspot_briefs`  
- `infra_prioritizer_us.hotspot_briefs_qc`

**Models**  
- `infra_prioritizer_us.text_embedding_005`  
- `infra_prioritizer_us.gemini_text`

**Indices (optional)**  
- `infra_prioritizer_us.tickets_ix` on `tickets_emb_base`


## 31. Cleanup (optional, commented for safety)

In [142]:
# %%bigquery --project bq-ai-hackathon-469420
# SET @@location = 'US';
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.tickets_emb_base`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_relaxed`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_matches_best`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_thresholds`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_terms`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_terms_emb`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_precision_at_threshold`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.seed_allowlist`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.cluster_geoms_v2`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.cluster_summary_v2`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.prioritized_hotspots_v2_qc`;
# DROP TABLE `bq-ai-hackathon-469420.infra_prioritizer_us.map_hotspots_v2`;
# DROP VIEW  `bq-ai-hackathon-469420.infra_prioritizer_us.ops_worklist_with_briefs`;
# DROP MODEL `bq-ai-hackathon-469420.infra_prioritizer_us.text_embedding_005`;
# DROP MODEL `bq-ai-hackathon-469420.infra_prioritizer_us.gemini_text`;

## 32. Appendix — risks, limits, next steps
- **Weak‑label noise.** E1 is a **lower bound**; we mitigate via per‑seed thresholds + QC gating.  
- **Geometric artifacts.** Convex hulls of colinear points become lines ⇒ we buffer (Sec. 18) to guarantee polygons.  
- **Cost guards.** 180‑day slice, `top_k`≤500, deterministic generation, ANN only with healthy coverage.  
- **Roadmap.** (1) expand seeds from centroid tickets; (2) budget‑aware routing (unit‑cost priors, travel time); (3) human‑in‑the‑loop curation UI for cluster edits.

> **Reviewer tip:** To demonstrate value quickly, export `map_hotspots_v2` to Geo Viz, overlay city basemap, and color by `priority_score`. This mirrors a simple crew dispatch view.